In [64]:
import pandas as pd
import boto3
import json
import time
import psycopg2

## Extracting values from the Config File

In [65]:
import configparser
config = configparser.ConfigParser()
with open('dwh.cfg') as fp:
    config.read_file(fp)

KEY                    = config.get('AWS','KEY')
SECRET                 = config.get('AWS','SECRET')

DWH_CLUSTER_TYPE       = config.get("DWH","DWH_CLUSTER_TYPE")
DWH_NUM_NODES          = config.get("DWH","DWH_NUM_NODES")
DWH_NODE_TYPE          = config.get("DWH","DWH_NODE_TYPE")

DWH_CLUSTER_IDENTIFIER = config.get("DWH","DWH_CLUSTER_IDENTIFIER")
DWH_DB                 = config.get("DWH","DWH_DB")
DWH_DB_USER            = config.get("DWH","DWH_DB_USER")
DWH_DB_PASSWORD        = config.get("DWH","DWH_DB_PASSWORD")
DWH_PORT               = config.get("DWH","DWH_PORT")

DWH_IAM_ROLE_NAME      = config.get("DWH", "DWH_IAM_ROLE_NAME")

(DWH_DB_USER, DWH_DB_PASSWORD, DWH_DB)

pd.DataFrame({"Param":
                  ["DWH_CLUSTER_TYPE", "DWH_NUM_NODES", "DWH_NODE_TYPE", "DWH_CLUSTER_IDENTIFIER", "DWH_DB", "DWH_DB_USER", "DWH_DB_PASSWORD", "DWH_PORT", "DWH_IAM_ROLE_NAME"],
              "Value":
                  [DWH_CLUSTER_TYPE, DWH_NUM_NODES, DWH_NODE_TYPE, DWH_CLUSTER_IDENTIFIER, DWH_DB, DWH_DB_USER, DWH_DB_PASSWORD, DWH_PORT, DWH_IAM_ROLE_NAME]
             })

,Param,Value
0,DWH_CLUSTER_TYPE,multi-node
1,DWH_NUM_NODES,4
2,DWH_NODE_TYPE,dc2.large
3,DWH_CLUSTER_IDENTIFIER,redshift-cluster
4,DWH_DB,redshift-cluster
5,DWH_DB_USER,dev
6,DWH_DB_PASSWORD,Edward123
7,DWH_PORT,5439
8,DWH_IAM_ROLE_NAME,redshift_role


In [66]:
import boto3

ec2 = boto3.resource('ec2',
                       region_name="us-west-2",
                       aws_access_key_id=KEY,
                       aws_secret_access_key=SECRET
                    )

s3 = boto3.resource('s3',
                       region_name="us-west-2",
                       aws_access_key_id=KEY,
                       aws_secret_access_key=SECRET
                   )

iam = boto3.client('iam',aws_access_key_id=KEY,
                     aws_secret_access_key=SECRET,
                     region_name='us-west-2'
                  )

redshift = boto3.client('redshift',
                       region_name="us-west-2",
                       aws_access_key_id=KEY,
                       aws_secret_access_key=SECRET
                       )

## Creating Functions

In [67]:
def create_iam_role():
    try:
        """creating a new IAM role for redshift"""
        dwhRole = iam.create_role(
            Path='/',
            RoleName=DWH_IAM_ROLE_NAME,
            AssumeRolePolicyDocument= json.dumps(
                {
                    'Statement':[{'Action': 'sts:AssumeRole', 'Effect': 'Allow', 'Principal': {'Service': 'redshift.amazonaws.com'}}]
                }
            )
       )

        """Attaching S3-read-only policy to that role"""

        iam.attach_role_policy(RoleName=DWH_IAM_ROLE_NAME,
                               PolicyArn="arn:aws:iam::aws:policy/AmazonS3ReadOnlyAccess"
                              )['ResponseMetadata']['HTTPStatusCode']

        
        """get the role ARN"""
        roleArn = iam.get_role(RoleName=DWH_IAM_ROLE_NAME)['Role']['Arn']

        return roleArn
    except Exception as e:
        print(e)



In [68]:
def create_redshift_cluster(iam_role):    
    try:
        response = redshift.create_cluster(        
            #HW
            ClusterType=DWH_CLUSTER_TYPE,
            NodeType=DWH_NODE_TYPE,
            NumberOfNodes=int(DWH_NUM_NODES),

            #Identifiers & Credentials
            DBName=DWH_DB,
            ClusterIdentifier=DWH_CLUSTER_IDENTIFIER,
            MasterUsername=DWH_DB_USER,
            MasterUserPassword=DWH_DB_PASSWORD,

            #Roles (for s3 access)
            IamRoles=[iam_role]  
        )
    except Exception as e:
        print(e)
    

In [69]:
def describe_redshift_cluster(): 
    myClusterProps = redshift.describe_clusters(ClusterIdentifier=DWH_CLUSTER_IDENTIFIER)['Clusters'][0]
    status = myClusterProps['ClusterStatus']
    print(status)
    
    while status != 'available':
        print("waiting for the cluster is up")
        time.sleep(10)
        status = redshift.describe_clusters(ClusterIdentifier=DWH_CLUSTER_IDENTIFIER)['Clusters'][0]['ClusterStatus']
        print(status)
    print('Cluster is up')
    myClusterProps = redshift.describe_clusters(ClusterIdentifier=DWH_CLUSTER_IDENTIFIER)['Clusters'][0]
    DWH_ENDPOINT = myClusterProps['Endpoint']['Address']
    DWH_VPC_ID = myClusterProps['VpcId']
    return (DWH_VPC_ID, DWH_ENDPOINT)
            

In [70]:
def authorize_ingest_sg(vpc_id):  
    try:
        vpc = ec2.Vpc(id=vpc_id)
        defaultSg = list(vpc.security_groups.all())[0]
        defaultSg.authorize_ingress(
            GroupName=defaultSg.group_name,
            CidrIp='0.0.0.0/0',
            IpProtocol='TCP',
            FromPort=int(DWH_PORT),
            ToPort=int(DWH_PORT)
        )
    except Exception as e:
        print(e)
        

## Executing Functions


In [72]:
role_arn = create_iam_role()
create_redshift_cluster(role_arn)
DWH_VPC_ID, DWH_ENDPOINT = describe_redshift_cluster()
authorize_ingest_sg(DWH_VPC_ID)
print(DWH_ENDPOINT)

creating
waiting for the cluster is up
creating
waiting for the cluster is up
creating
waiting for the cluster is up
creating
waiting for the cluster is up
creating
waiting for the cluster is up
creating
waiting for the cluster is up
creating
waiting for the cluster is up
creating
waiting for the cluster is up
creating
waiting for the cluster is up
creating
waiting for the cluster is up
creating
waiting for the cluster is up
creating
waiting for the cluster is up
creating
waiting for the cluster is up
creating
waiting for the cluster is up
creating
waiting for the cluster is up
creating
waiting for the cluster is up
available
Cluster is up
An error occurred (InvalidPermission.Duplicate) when calling the AuthorizeSecurityGroupIngress operation: the specified rule "peer: 0.0.0.0/0, TCP, from port: 5439, to port: 5439, ALLOW" already exists
redshift-cluster.cmfrhuz96ffg.us-west-2.redshift.amazonaws.com


## Important

In [76]:
# Copy the DWH_ENDPOINT value go to the dwh.cfg to paste it to the Host key under [CLUSTER]
# Copy the role_arn value and go to the dwh.cfg to paste in to the ARN key under [IAM_ROLE]
print(DWH_ENDPOINT)
print(role_arn)

redshift-cluster.cmfrhuz96ffg.us-west-2.redshift.amazonaws.com
arn:aws:iam::709891834787:role/redshift_role
